# Targeted poisoning attack on MNIST dataset

## Import Libraries

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
from matplotlib import pyplot as plt
from sklearn.metrics import classification_report

import collections
from tqdm import tqdm


import random
from tqdm.notebook import tqdm
import copy

## Parameters declaration

In [2]:
n_clients = 60
n_test_clients = 60

n_train_dataset_epochs = 6
n_test_dataset_epochs = 3
n_train_epochs = 30
batch_size_train = 20
batch_size_test = 4

client_learning_rate = 0.02
server_learning_rate = 1.5


hidden_units = 256
dropout = 0.1

mal_users_percentage = 0.2
# todo: could also be a list of values
target_value = 3
poisoned_value = 8



## Dataset Loading and manipulation

In [3]:
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()

In [4]:
example_dataset = emnist_train.create_tf_dataset_for_client(
    emnist_train.client_ids[45])

test_dataset = emnist_test.create_tf_dataset_for_client(
    emnist_train.client_ids[45])

print(emnist_train.client_ids[0:10])
print(emnist_test.client_ids[0:10])
cnt_train = 0
cnt_test = 0

for item in example_dataset:
    cnt_train += 1 

for item in test_dataset:
    cnt_test += 1


print(cnt_train)
print(cnt_test)

['f0000_14', 'f0001_41', 'f0005_26', 'f0006_12', 'f0008_45', 'f0011_13', 'f0014_19', 'f0016_39', 'f0017_07', 'f0022_10']
['f0000_14', 'f0001_41', 'f0005_26', 'f0006_12', 'f0008_45', 'f0011_13', 'f0014_19', 'f0016_39', 'f0017_07', 'f0022_10']
93
11


### Preprocessing and organizing dataset

In [5]:
shuffle_buffer = 100
# todo change??
prefetch_buffer = 10

In [6]:
def batch_format(element):
    # flatten the images
    return collections.OrderedDict(
        x = tf.reshape(element['pixels'], [-1, 28, 28]),
        y = tf.reshape(element['label'], [-1, 1]))

def preprocess(dataset, train):
    if train == True:
        dataset = dataset.repeat(n_train_dataset_epochs)
    else:
        dataset = dataset.repeat(n_test_dataset_epochs)
        
    dataset = dataset.shuffle(shuffle_buffer, seed = 1)
    if train == True:
        dataset = dataset.batch(batch_size_train)
    else: 
        dataset = dataset.batch(batch_size_test)
        
    dataset = dataset.map(batch_format)
    dataset = dataset.prefetch(prefetch_buffer)

    return dataset

In [7]:
def poison_dataset(dataset, target_honest, target_mal):
    # parse the dataset
    for batch in dataset:
        # get the labels of each batch and convert to numpy array
        labels = batch['y'].numpy()
        # itterate through each label
        for i,y in enumerate(labels):
            # if we find the target label	
            if y == target_honest:
                labels[i] = target_mal
        batch['y'] = tf.convert_to_tensor(labels, dtype = tf.float32)
        # print(batch['y'])
    # return the malicious dataset
    return dataset

In [8]:
## testing the preprocessing function

example_dataset = emnist_train.create_tf_dataset_for_client(
    emnist_train.client_ids[0])
     
preprocessed_example_dataset = preprocess(example_dataset, True)

In [9]:
def make_federated_data(client_data, client_ids, train, mal_users_percentage=0):
    target_honest = 3
    target_mal = 8
    fed_data = []
    for id in client_ids:
        preprocessed_dataset = preprocess(client_data.create_tf_dataset_for_client(id), train)
        prob = random.random()
        # mal% of the users are malicious
        if prob < mal_users_percentage:  
            preprocessed_dataset = poison_dataset(preprocessed_dataset, target_honest, target_mal)              

        fed_data.append(preprocessed_dataset)

    return fed_data

In [10]:
# print(emnist_test.client_ids[0])
data = make_federated_data(emnist_test, [emnist_train.client_ids[0]], train=False, mal_users_percentage=1)

## Model creation

In [11]:
def create_model():
      return tf.keras.models.Sequential([
      tf.keras.layers.Reshape(input_shape=(28,28,1), target_shape=(28,28,1)),
      tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dropout(dropout),
      tf.keras.layers.Dense(10, activation='softmax')
  ])

In [12]:
def mnist_model():
    keras_model = create_model()
    return tff.learning.models.from_keras_model(
        keras_model,
        input_spec = preprocessed_example_dataset.element_spec,
        loss = tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics = [tf.keras.metrics.SparseCategoricalAccuracy()])     

## Training

In [13]:
training_process = tff.learning.algorithms.build_weighted_fed_avg(
    mnist_model,
    client_optimizer_fn = lambda: tf.keras.optimizers.SGD(learning_rate = client_learning_rate),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate = server_learning_rate))

In [14]:
train_state = training_process.initialize()

In [15]:
# build the process to have the model's architecture
evaluation_process = tff.learning.algorithms.build_fed_eval(mnist_model)

# initialize the state of the evaluation
evaluation_state = evaluation_process.initialize()
sample_test_clients = emnist_test.client_ids[0:n_test_clients]


# step = 16
# sample_test_clients = [emnist_train.client_ids[x] for x in range (0, n_clients * step, step)]

# test the model with the test data
# question: selection of clients during training??
federated_test_data = make_federated_data(emnist_test, sample_test_clients, train=False)

In [16]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

training_loss = []
training_acc = []
eval_loss = []
eval_acc = []

clients = emnist_train.client_ids[0:n_clients]
step = 16
clients = [emnist_train.client_ids[x] for x in range (0, n_clients * step, step)]

#note: slow to converge with random clients, makes sense

for epoch in range(1, n_train_epochs):

    print('Epoch: {:2d}'.format(epoch))
    # client selection, random, chosen from the first 100 clients
    clients = random.sample(emnist_train.client_ids, n_clients)

    federated_train_data = make_federated_data(emnist_train, clients, train=True)
    federated_test_data = make_federated_data(emnist_test, sample_test_clients, train=False)
    
    
    # run a next on the training process to train the model
    result = training_process.next(train_state, federated_train_data)
    # update the model's state and get access to the metrics
    train_state = result.state
    train_metrics = result.metrics
    # print the training metrics
    training_acc.append(train_metrics['client_work']['train']['sparse_categorical_accuracy'])
    training_loss.append(train_metrics['client_work']['train']['loss'])

    print('Training accuracy: {:.3f}, Training loss: {:.3f}'.format(training_acc[-1], training_loss[-1]))

    # evaluate the model with test data

    # get weights from the trainged model
    model_weights = training_process.get_model_weights(train_state)
    # update the evaluation state with them
    evaluation_state = evaluation_process.set_model_weights(evaluation_state, model_weights)
    # run a next() to evaluate the model
    evaluation_output = evaluation_process.next(evaluation_state, federated_test_data)

    # get access to the evaluation metrics
    eval_metrics = evaluation_output.metrics['client_work']['eval']['total_rounds_metrics']

    eval_acc.append(eval_metrics['sparse_categorical_accuracy'])
    eval_loss.append(eval_metrics['loss'])
    # print the training metrics
    print('Testing accuracy: {:.3f}, Testing loss: {:.3f}\n\n'.format(eval_acc[-1], eval_loss[-1]))

Epoch:  1
Training accuracy: 0.127, Training loss: 2.300
Testing accuracy: 0.164, Testing loss: 2.279


Epoch:  2
Training accuracy: 0.210, Training loss: 2.252
Testing accuracy: 0.421, Testing loss: 2.240


Epoch:  3
Training accuracy: 0.342, Training loss: 2.207
Testing accuracy: 0.551, Testing loss: 2.187


Epoch:  4
Training accuracy: 0.496, Training loss: 2.142
Testing accuracy: 0.604, Testing loss: 2.111


Epoch:  5
Training accuracy: 0.576, Training loss: 2.056
Testing accuracy: 0.636, Testing loss: 2.007


Epoch:  6
Training accuracy: 0.621, Training loss: 1.927
Testing accuracy: 0.661, Testing loss: 1.868


Epoch:  7
Training accuracy: 0.695, Training loss: 1.780
Testing accuracy: 0.711, Testing loss: 1.692


Epoch:  8
Training accuracy: 0.770, Training loss: 1.555
Testing accuracy: 0.712, Testing loss: 1.494


Epoch:  9
Training accuracy: 0.798, Training loss: 1.354
Testing accuracy: 0.731, Testing loss: 1.295


Epoch: 10
Training accuracy: 0.808, Training loss: 1.188
Testing

In [21]:
y_true = []
y_pred = []

for batch in evaluation_output:
    print(batch)
    # pred = np.argmax(batch)

#     print(batch['y'])




LearningAlgorithmState(global_model_weights=ModelWeights(trainable=[array([[[[ 1.40045181e-01,  2.50327855e-01, -7.52348900e-02,
           3.43430117e-02,  6.40342608e-02, -4.92724478e-02,
           4.24161017e-01, -7.60460598e-03,  1.05704226e-01,
           3.49419951e-01,  9.68406349e-02,  1.53103480e-02,
           2.81466246e-02, -3.42142075e-01,  3.04258227e-01,
          -3.59155506e-01, -2.14422613e-01,  3.75122875e-01,
          -4.79744703e-01,  3.94661240e-02,  5.00482041e-03,
           2.27420658e-01,  3.02444518e-01, -6.21368647e-01,
          -9.77644771e-02,  3.77750456e-01,  1.17097057e-01,
          -2.54666880e-02,  4.58100349e-01, -1.69613868e-01,
          -9.31188688e-02,  1.35494262e-01]],

        [[-2.33093992e-01,  2.57363975e-01,  6.58216886e-05,
          -4.94578987e-01,  3.52748781e-01, -7.24928752e-02,
           8.47739354e-02,  1.08471755e-02,  9.80660468e-02,
           6.19769357e-02,  1.51732504e-01, -7.17567354e-02,
          -3.15265000e-01, -4.4

In [ ]:
emnist_train
# clients = emnist_train.client_ids[0:n_clients]
# print(clients)

In [ ]:
print(evaluation_output.metrics)

In [ ]:
epochs = [x for x in range(1, n_train_epochs)]
plt.plot(epochs, training_acc)
plt.plot(epochs, eval_acc)
plt.show()

In [ ]:
epochs = [x for x in range(1, n_train_epochs)]
plt.plot(epochs, training_loss)
plt.plot(epochs, eval_loss)
plt.show()

In [11]:
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()

accs = []
losses = []

example_dataset = emnist_train.create_tf_dataset_for_client(
    emnist_train.client_ids[0])

preprocessed_example_dataset = preprocess(example_dataset, True)
 
     
# build the process to have the model's architecture
evaluation_process = tff.learning.algorithms.build_fed_eval(mnist_model)

# initialize the state of the evaluation
sample_test_clients = emnist_test.client_ids[0:n_test_clients]

federated_test_data = make_federated_data(emnist_test, sample_test_clients, train=False)


n_train_epochs = 20

for i in [0, 5, 6, 7]:
    evaluation_state = evaluation_process.initialize()

    eval_acc = []
    eval_loss = []
    
    mal_users_percentage = i / 10
    
    emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()

    training_process = tff.learning.algorithms.build_unweighted_fed_avg(
        mnist_model,
        client_optimizer_fn = lambda: tf.keras.optimizers.SGD(learning_rate = client_learning_rate),
        server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate = server_learning_rate))

    train_state = training_process.initialize()
    print("Model with " + str(i) + "% mal clients")
    for epoch in tqdm(range(0, n_train_epochs)):
        
        # client selection, random, chosen from the first 100 clients
        # clients = random.sample(emnist_train.client_ids, n_clients)
        clients = emnist_train.client_ids[0:n_clients]

        federated_train_data = make_federated_data(emnist_train, clients, train=True, mal_users_percentage=mal_users_percentage)      
        
        # run a next on the training process to train the model
        result = training_process.next(train_state, federated_train_data)
        # update the model's state and get access to the metrics
        train_state = result.state
                
        # get weights from the trainged model
        model_weights = training_process.get_model_weights(train_state)
        # update the evaluation state with them
        evaluation_state = evaluation_process.set_model_weights(evaluation_state, model_weights)
        # run a next() to evaluate the model
        evaluation_output = evaluation_process.next(evaluation_state, federated_test_data)

        # get access to the evaluation metrics
        eval_metrics = evaluation_output.metrics['client_work']['eval']['total_rounds_metrics']

        eval_acc.append(eval_metrics['sparse_categorical_accuracy'])
        eval_loss.append(eval_metrics['loss'])        

    accs.append(eval_acc)
    losses.append(eval_loss)


Model with 0% mal clients


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


AttributeError: 'FloatProgress' object has no attribute 'style'

In [ ]:
epochs = [x for x in range(0, n_train_epochs)]
for cnt, model_acc in enumerate(accs):

    plt.plot(epochs, model_acc, label=str(cnt * 10) + "% of mal users")

plt.legend()
plt.show()